# Imports

In [1]:
import os
import pandas
import bs4
import math
import datetime
import random
from numpy.random import randint as ri

# Helper functions

### Generate random text

In [2]:
lorem = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.'
lorem_parts = lorem.split()

def return_random_text(lower, upper):
    # Random number of words between 0-30
    num_words = ri(lower, upper)
    words = [lorem_parts[ri(0, len(lorem_parts))] for num in range(num_words)]
    paragraph = ' '.join(words)
    return paragraph.capitalize()

### Generate random tags

In [3]:
test_tags = ['blue', 'red', 'yellow', 'purple', 'grey', 'white', 'black', 'teal', 'orange', 'silver', 'pink', 'brown']

def generate_tags():
    tags = ''
    tag_list = [test_tags[ri(0, len(test_tags)-1)] for tag in range(ri(2,8))]
    tags = ','.join(tag_list)
    return tags

### Generate lists of lists (for pagination)

In [4]:
def cluster(sequence, per_group):
    steps = list(range(0, len(sequence), per_group))
    groups = []
    for index, step in enumerate(steps):
        temp = list()
        for n in range(per_group):
            absolute_index = (index + 1) * per_group - per_group + n
            if absolute_index < len(sequence):
                temp.append(sequence[step])
                step = step + 1
        groups.append(temp)
    return groups

# Generator settings
- data paths
- fonts
- bootstrap colour theming
- pagination options

### Paths

In [5]:
data_path = './data/pokemon_svg/'
relative_data_path = '../data/pokemon_svg/'
html_path = './html/'

### Site Fonts

In [6]:
google_fonts = {"bootstrap":{"link":'',
                            "specify":'',
                            "url":None},
                
                "raleway":{"link":'<link href="https://fonts.googleapis.com/css?family=Raleway" rel="stylesheet">',
                          "specify":"font-family: 'Raleway', sans-serif;",
                          "url":"https://fonts.google.com/specimen/Raleway?selection.family=Raleway"},
                
                "open_sans":{"link":'<link href="https://fonts.googleapis.com/css?family=Open+Sans" rel="stylesheet">',
                        "specify":"font-family: 'Open Sans', sans-serif;",
                       "url":"https://fonts.google.com/specimen/Open+Sans?selection.family=Open+Sans"},
                
                "roboto":{"link":'<link href="https://fonts.googleapis.com/css?family=Roboto" rel="stylesheet">',
                         "specify":"font-family: 'Roboto', sans-serif;",
                         "url":"https://fonts.google.com/specimen/Roboto?selection.family=Roboto"},
                
                "ubuntu":{"link":'<link href="https://fonts.googleapis.com/css?family=Ubuntu" rel="stylesheet"> ',
                         "specify":"font-family: 'Ubuntu', sans-serif;",
                         "url": "https://fonts.google.com/specimen/Ubuntu?selection.family=Ubuntu"}}

In [7]:
selected_font = 'ubuntu'

set_link = google_fonts[selected_font]['link']
set_style = google_fonts[selected_font]['specify']
print('Font has been set to {}'.format(selected_font))

Font has been set to ubuntu


### Site Colours

In [8]:
jumbotron_background = '#1d91c0'
jumbotron_foreground = '#FFFFFF'
outline_button = 'dark'
header_foreground = '#44494a'
header_background = '#FFFFFF'
navbar_background = '#414242'
footer_background = '#414242'
pill_background = '#1d91c0'
pill_foreground  = '#FFFFFF'

### Site pagination

In [9]:
items_per_page = 12

svg = True

if svg == True:
    visual_tag_start = '<object class = "p-1" height=100% width=100% type="image/svg+xml" data="'
    visual_tag_end = '">Your browser does not support SVG</object>'
else:
    visual_tag_start = '<img class="card-img-top p-3" src="'
    visual_tag_end = '" alt="Card image cap">'

### Set enabled/disabled variables used to switch buttons between enabled and disabled modes

In [10]:
nothing = ''
disabled = 'disabled'

## Read in data

### Read filenames from data_path variable

In [11]:
all_filenames = [file for file in os.listdir(data_path) if 'DS_Store' not in file]

### Store filenames in dataframe

In [12]:
data = pandas.DataFrame()
data["filename"] = pandas.Series(all_filenames)

### Create tags for testing purposes
- Create list of possible tags, this is arbitrary
- For each row in dataset containing filenames, calculate a random number (number of tags for this particular file), then for each increment of this random number assign a random tag from test_tags list

In [13]:
for index in range(data.shape[0]):
    data.loc[index, "tag"] = generate_tags()

### Extract unique tags from dataframe

In [14]:
tag_string = ','.join(data['tag'].tolist())

In [15]:
tags_including_dupes = [tag for tag in tag_string.split(',') if len(tag)> 0]

In [16]:
no_dupes = sorted(list(set(tags_including_dupes)))

### Create dictionary to store indices for each tag
- Create the dictionary to store each unique tag, tags will be used to filter items
- Iterate through and its associated indices (each index for each filename associated with that tag)
- Finally add an 'index' key with all indices, this will be used as an unfiltered set of pages
- Preview dictionary to validate tag/index assignment

In [17]:
from collections import defaultdict

tag_storage = defaultdict(list)

for tag in no_dupes:
    for index in range(data.shape[0]):
        current_tag = data.loc[index, "tag"]
        if tag in current_tag:
            tag_storage[tag].append(index)

tag_storage["index"] = list(data.index)

print(tag_storage)

defaultdict(<class 'list'>, {'black': [0, 1, 5, 10, 11, 13, 17, 19, 20, 25, 26, 28, 31, 33, 40, 43, 44, 45, 46, 47, 50, 53, 54, 60, 64, 65, 72, 75, 80, 87, 89, 90, 93, 94, 96], 'blue': [0, 2, 3, 5, 9, 19, 26, 30, 34, 48, 49, 51, 54, 62, 63, 67, 70, 73, 74, 76, 79, 82, 84, 87, 92, 95, 98, 99], 'grey': [3, 4, 6, 8, 10, 12, 13, 15, 16, 17, 24, 30, 31, 32, 41, 45, 48, 52, 56, 57, 61, 63, 64, 65, 66, 67, 72, 77, 79, 80, 83, 84, 85, 86, 97], 'orange': [5, 10, 13, 16, 19, 21, 22, 27, 28, 33, 35, 38, 39, 40, 44, 45, 46, 49, 51, 55, 58, 63, 70, 71, 74, 84, 90, 92], 'pink': [16, 18, 26, 31, 32, 35, 38, 47, 48, 49, 50, 52, 55, 57, 59, 64, 66, 67, 71, 83, 86], 'purple': [2, 3, 7, 11, 16, 20, 23, 28, 32, 33, 38, 41, 42, 45, 46, 47, 49, 51, 52, 61, 62, 64, 67, 68, 70, 73, 79, 83, 86, 88, 91, 92, 93, 95, 98], 'red': [0, 4, 11, 13, 15, 20, 23, 34, 42, 45, 50, 53, 65, 68, 71, 74, 76, 78, 81, 82, 83, 85, 87, 88, 91, 92, 95, 96, 99], 'silver': [2, 6, 7, 14, 18, 19, 22, 25, 27, 29, 36, 37, 44, 46, 50, 55,

## Templates

### Jumbotron

In [18]:
jumbotron_home = '''
<div class="jumbotron jumbotron-fluid text-center m-0" style = "color: '''+jumbotron_foreground+'''; background-color:'''+jumbotron_background+''';">
      <div class="container">
        <h1 class="display-2">
          Daniels Generated Album
        </h1>
        <p class="lead">
          Something short and leading about the collection below—its contents, the creator, etc. Make it short and sweet, but not too short so folks don't simply skip over it entirely.
        </p>
        <p>
          <a class="btn btn-outline-light my-2" href="#">
          Primary
          </a>
          <a class="btn btn-outline-light my-2" href="#">
          Secondary
          </a>
        </p>
      </div>
    </div>
'''

### Pagination - Variable

In [19]:
pagination = '<div class = "text-center pb-4 text-muted bg-light">\
  <div class = "btn-group">\
    <a href = "{tag}{first}.html" class = "btn btn-outline-'+outline_button+' {class1}">First</a>\
    <a href = "{tag}{prev}.html" class = "btn btn-outline-'+outline_button+' {class1}">Previous</a>\
    <a class = "btn btn-outline-'+outline_button+' active">Page {page}</a>\
    <a href = "{tag}{next_}.html" class = "btn btn-outline-'+outline_button+' {class2}">Next</a>\
    <a href = "{tag}{last}.html"class = "btn btn-outline-'+outline_button+' {class2}">Last</a>\
  </div>\
</div>'

pagination2 = '''
<div class = "text-center pb-4 text-muted bg-light">
  <div class = "btn-group">
    <a class = "btn btn-outline-'+outline_button+' active">Page {page}</a>
  </div>
</div>
'''

### Card 

In [20]:
card = '''
<section class = "col-lg-4 col-md-6 mb-4">\
    <div class="card">\
        <div class = "card-header p-3" style = "background-color: {header_background};">\
            <h5 class = "p-0 m-0">Source: {filename}</h5>\
        <div class = "card-subtitle p-0 m-0 text-muted">{subtitle}</div>\
        </div>\
        {visual_tag_start}{relative_data_path}{filename}{visual_tag_end}\
        <div class="card-body">\
        <p class="card-text">{text}</p>\
        <div class="btn-group p-0 m-0">\
        <a class="btn btn-sm btn-outline-{outline_button}" href="{relative_data_path}{filename}" role="button">View</a>\
        <a class="btn btn-sm btn-outline-{outline_button}" href ="{relative_data_path}{filename}" download = "{filename}" role="button">Download</a>\
        </div>\
        </div>\
        <div class = "p-3 bg-light">\
        {tags_html_block}\
        </div>\
    </div>\
</section>
'''

### Skeleton

In [21]:
skeleton = '''
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8"/>
    <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
    <meta content="" name="description"/>
    <meta content="" name="author"/>
    <link href="../../../../favicon.ico" rel="icon"/>
    <title>
      Daniels Generated Gallery
    </title>
    <!-- Bootstrap CSS -->
    ''' + set_link + '''
    <link crossorigin="anonymous" href="https://stackpath.bootstrapcdn.com/bootstrap/4.1.1/css/bootstrap.min.css" integrity="sha384-WskhaSGFgHYWDcbwN70/dfYBj47jz9qbsMId/iRN3ewGhXQFZCSftd1LZCfmhktB" rel="stylesheet"/>
    <!-- Custom styles for this template -->
    <link href="album.css" rel="stylesheet"/>
  </head>
  <body class = "bg-light" style = "'''+set_style+'''">
    <header style = "background-color:'''+navbar_background+''';">
       <div class="container">
        <nav class="navbar navbar-expand-lg navbar-dark py-1" style = "background-color:'''+navbar_background+''';">
         <a class="navbar-brand" href="#">
          <img alt="" class="d-inline-block" height="36" src="../logo/analytics.svg" width="36"/>
         </a>
         <button aria-controls="navbarColor03" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler collapsed" data-target="#navbarColor03" data-toggle="collapse" type="button">
          <span class="navbar-toggler-icon">
          </span>
         </button>
         <div class="navbar-collapse collapse" id="navbarColor03" style="">
          <ul class="navbar-nav mr-auto">
           <li class="nav-item active">
            <a class="nav-link" href="index0.html">
             Home
             <span class="sr-only">
              (current)
             </span>
            </a>
           </li>
           <li class="nav-item">
            <a class="nav-link" href="#">
             About
            </a>
           </li>
           <li class="nav-item">
            <a class="nav-link" href="#">
             Donate
            </a>
           </li>
           <li class="nav-item">
            <a class="nav-link" href="#">
             Community
            </a>
           </li>
           <li class="nav-item">
            <a class="nav-link" href="#">
             Contact Us
            </a>
           </li>
          </ul>
         </div>
    </nav>
   </div>
  </header>
    
    
    {}
    
    
    
    <div class="album bg-light">
      <div class="container">
        <div class="row mt-3">
          
          {}
        
        
        </div>
      </div>
    </div>
    
    {}
    
    <footer class="text-light py-3" style = "background-color: '''+footer_background+''';">
      <div class="container">
        <span>&copy; Copyright DanielsGenerator</span>
        <span class="float-right">
        <a href="#">
        Back to top
        </a>
        </span>
      </div>
    </footer>
    <!-- Optional JavaScript -->
    <!-- jQuery first, then Popper.js, then Bootstrap JS -->
    <script crossorigin="anonymous" integrity="sha384-q8i/X+965DzO0rT7abK41JStQIAqVgRVzpbzo5smXKp4YfRvH+8abtTE1Pi6jizo" src="https://code.jquery.com/jquery-3.3.1.slim.min.js"></script>
    <script crossorigin="anonymous" integrity="sha384-ZMP7rVo3mIykV+2+9J3UJ46jBk0WLaUAdn689aCwoqbBJiSnjAK/l8WvCWPIPm49" src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.14.3/umd/popper.min.js"></script>
    <script crossorigin="anonymous" integrity="sha384-smHYKdLADwkXOn1EmN1qk/HfnUcbVRZyYmZ4qpPea6sjB/pTJ0euyQp0Mk8ck+5T" src="https://stackpath.bootstrapcdn.com/bootstrap/4.1.1/js/bootstrap.min.js"></script>
  </body>
</html>
'''

# Before Generation - Delete all previous `.html`

In [22]:
[os.remove(html_path + file) for file in os.listdir(html_path) if '.html' in file]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

# Generate the website


- **ITERATE THROUGH EACH TAG `tag_storage` (the higest level)**
    - Iterate through every `tag` in `tag_storage` dictionary
    - Work out `total_items`, `excess_items`, `total_pages`
    - Calculate `nested_page_indices`
    

- **ITERATE THROUGH EACH PAGE IN `tag_level_items`** (second level, represents html pages, containing `items_per_page` items)
    - Calculate `previous_index` and `next_index`, used for pagination linking
    - Calculate pagination html
    - Calculate which jumbotron variable to use



- **ITERATE THROUGH EACH FILE IN `page_items`** (lowest level, represents filenames)
    - Calculate `filename` and `filetags` variables
    - Calculate html block using `filetags` variable
    - Calculate group of bootstrap cards for body (`cards`)

In [24]:
#ITERATE THROUGH EACH TAG IN tag_storage

for tag, file_list in tag_storage.items():
    
    print('Processing tag_storage key:', tag)
    tag_level_items = cluster(file_list, items_per_page)
    
    #ITERATE THROUGH EACH PAGE IN tag_level_items

    for page_index, page_items in enumerate(tag_level_items):
    
        # WORK OUT INDEX FOR PAGES LATERALLY
        previous_index = page_index - 1
        next_index = page_index + 1
        total_pages = len(tag_level_items)
        last_index = len(tag_level_items)-1
        #PAGINATION
        if total_pages == 1:
            pagination_final = pagination2.format(page=page_index)
        else:
            if page_index == 0:
                pagination_final = pagination.format(tag=tag, first=0, prev=previous_index, page = page_index, 
                                                     next_ = next_index, last = last_index, class1 = disabled, class2 = nothing)
            elif page_index == last_index:
                pagination_final = pagination.format(tag=tag, first=0, prev=previous_index, page = page_index, 
                                                     next_ = next_index, last = last_index, class1 = nothing, class2 = disabled)
            else:
                pagination_final = pagination.format(tag=tag, first=0, prev=previous_index, page = page_index, 
                                                     next_ = next_index, last = last_index, class1 = nothing, class2 = nothing)

        if page_index == 0:
            jumbotron_final = jumbotron_home
        else:
            jumbotron_final = ''
            
        # CREATE EMPTY CARDS 
        cards = ''
        
        # FILE LEVEL LOOP
        for index2, file_index in enumerate(page_items):
            
            filename = data.loc[file_index, 'filename']
            filetags = data.loc[file_index, 'tag']

            tags_split = filetags.split(',')
            tags_html_block = ''

            for tag_name in tags_split:
                tags_html_block = tags_html_block + \
                '<a href = "'+tag_name+'0.html" class = "badge badge-pill" style = "color:'+ pill_foreground +' ;background-color:'+ pill_background +';">'+tag_name+'</a>'
            
            # Generate random text
            text = return_random_text(30,40)
            subtitle = return_random_text(6,8)
            
            cards = cards + card.format(filename=filename, header_background=header_background, subtitle=subtitle,
                       visual_tag_end=visual_tag_end, visual_tag_start=visual_tag_start, text=text, 
                       outline_button=outline_button, relative_data_path=relative_data_path, tags_html_block=tags_html_block)
            
        # BUILD FULL HTML CODE ON PAGE LEVEL
        full = skeleton.format(jumbotron_final, cards, pagination_final)

        # PRETTIFY/FORMAT THE FULL HTML CODE
        full_soup = bs4.BeautifulSoup(full, 'lxml')
        full_pretty = full_soup.prettify()

        #EXPORT HTML
        export_path = './html/{}{}.html'.format(tag, page_index)
        
        with open(export_path, "w") as f:
            f.write(full_pretty)
            f.close()
            print('\tFile successfully written to', export_path)

Processing tag_storage key: black
	File successfully written to ./html/black0.html
	File successfully written to ./html/black1.html
	File successfully written to ./html/black2.html
Processing tag_storage key: blue
	File successfully written to ./html/blue0.html
	File successfully written to ./html/blue1.html
	File successfully written to ./html/blue2.html
Processing tag_storage key: grey
	File successfully written to ./html/grey0.html
	File successfully written to ./html/grey1.html
	File successfully written to ./html/grey2.html
Processing tag_storage key: orange
	File successfully written to ./html/orange0.html
	File successfully written to ./html/orange1.html
	File successfully written to ./html/orange2.html
Processing tag_storage key: pink
	File successfully written to ./html/pink0.html
	File successfully written to ./html/pink1.html
Processing tag_storage key: purple
	File successfully written to ./html/purple0.html
	File successfully written to ./html/purple1.html
	File successful